In [68]:
# Set working directory
import os
os.chdir('C:/.../Ames-House-Prices')

In [69]:
import pandas as pd

In [70]:
data = pd.read_csv('train.csv') # hd = shorthand for housing data
data.head() # view the first 5 rows of the data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [71]:
cat_columns = data.select_dtypes(['object']).columns
cat_columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [72]:
data[cat_columns] = data[cat_columns].apply(lambda x: x.astype('category'))

In [73]:
for col in ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 
            'GarageQual', 'PoolQC']:
    data[col] = data[col].astype('category')
data.dtypes

Id                  int64
MSSubClass          int64
MSZoning         category
LotFrontage       float64
LotArea             int64
Street           category
Alley            category
LotShape         category
LandContour      category
Utilities        category
LotConfig        category
LandSlope        category
Neighborhood     category
Condition1       category
Condition2       category
BldgType         category
HouseStyle       category
OverallQual         int64
OverallCond         int64
YearBuilt           int64
YearRemodAdd        int64
RoofStyle        category
RoofMatl         category
Exterior1st      category
Exterior2nd      category
MasVnrType       category
MasVnrArea        float64
ExterQual        category
ExterCond        category
Foundation       category
                   ...   
BedroomAbvGr        int64
KitchenAbvGr        int64
KitchenQual      category
TotRmsAbvGrd        int64
Functional       category
Fireplaces          int64
FireplaceQu      category
GarageType  

In [74]:
def nearZeroVarPreds(df):
    """ This function applies the near-zero variance rules in Kuhn, Johnson's Applied Predictive Modeling textbook
    to categorical variables in a pandas dataframe. It uses a 10% threshold for fraction of unique values to total
    number of records in the data, and a ratio of 20 for the proportion of the most frequently occuring category to
    the ratio of the second most frequently occuring category.
    [Near-zero variance means that the] fraction of unique values over the sample size is low (say 10%) [...] 
    [and the] ratio of the frequency of the most prevalent value to the frequency of the second most prevalent 
    value is large (say around 20). If both of these criteria are true and the model in question is susceptible 
    to this type of predictor, it may be advantageous to remove the variable from the model.
    -- Kuhn, M., & Johnson, K. (2013). Applied predictive modeling, New York, NY: Springer.
    See also: https://stats.stackexchange.com/questions/145602/justification-for-feature-selection-by-removing-predictors-with-near-zero-varian)
    """
    nearZeroVarPreds = [] # initialize a list to hold the names of the near zero variance categorical variables
    for column in df: # loops through each column in the dataframe
        if df[column].dtype.name == 'category': # only applies the following to categorical variables
            fractionOfUniqValues = len(df[column].cat.categories) / df.shape[0] # fraction of unique categories to total number of records in the data
            colValCounts = data[column].cat.codes.value_counts().tolist() # returns values counts, sorted, as a list
            if len(colValCounts) > 1: # only applies the following to variables that have more than one category
                ratioOfCatFreqs = colValCounts[0] / colValCounts[1] # ratio of the frequency of the top most occuring category to the second most frequently occuring category
            else:
                ratioOfCatFreqs = 100 # default to a number above 20 for variables with only one category
            if fractionOfUniqValues < .10 and ratioOfCatFreqs >= 20: # condition for a near-zero variance variable, as defined above
                nearZeroVarPreds.append(column) # add the variable name to the list if it meets the criteria above
    return(nearZeroVarPreds) # return the list of variable names that are near-zero variance

In [75]:
variable_list = nearZeroVarPreds(data)

In [76]:
variable_list

['Street',
 'Alley',
 'LandContour',
 'Utilities',
 'LandSlope',
 'Condition2',
 'RoofMatl',
 'BsmtCond',
 'BsmtFinType2',
 'Heating',
 'Functional',
 'PoolQC',
 'MiscFeature']

In [77]:
def removeNearZeroVar(df, variable_list):
    newDF = df.copy()
    for var in variable_list:
        newDF = newDF.drop(var, 1)
    return(newDF)

In [78]:
newData = removeNearZeroVar(data, variable_list)

In [79]:
print(data.shape)
print(newData.shape)

(1460, 81)
(1460, 68)


In [80]:
data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [81]:
newData.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'LotShape',
       'LotConfig', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'Fence', 'MiscVal', 'MoSold', 'YrSold',
       'SaleType'